In [ ]:
import datasets

In [1]:
from datasets import Dataset

# Example documents
documents = [
    {"text": "The quick brown fox jumps over the lazy dog."},
    {"text": "All work and no play makes Jack a dull boy."},
    {"text": "To be, or not to be, that is the question."},
    {"text": "Budumba - this is a rare creature that inhabits the forests of the Northern Lands. It is named after the Native Americans who named it 'Budumba' because of its large head. It uses it's head to crack rocks and break up large trees to find food. It is a common sight in the forest."}
]

# Create a dataset from the documents
dataset = Dataset.from_dict({"documents": [doc["text"] for doc in documents]})

print(dataset)

c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['documents'],
    num_rows: 4
})


In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch
with torch.set_grad_enabled(False):
    ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

    ds = dataset.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["documents"], return_tensors="pt"))[0][0].numpy()})
    ds.add_faiss_index(column="embeddings")

c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (

In [2]:
train_test_split = dataset.train_test_split(test_size=0.2)

print(train_test_split)

DatasetDict({
    train: Dataset({
        features: ['documents'],
        num_rows: 2
    })
    test: Dataset({
        features: ['documents'],
        num_rows: 1
    })
})


In [3]:
ds.save_faiss_index('embeddings', 'my_index.faiss')
ds.drop_index("embeddings")
ds.save_to_disk("my_dataset")



Saving the dataset (1/1 shards): 100%|██████████| 4/4 [00:00<00:00, 1333.11 examples/s]


In [11]:
dts = Dataset.load_from_disk("my_dataset")
dts.load_faiss_index('embeddings', 'my_index.faiss')
